In [ ]:
# Cell 1: تثبيت المتطلبات الأساسية
!pip install ultralytics==8.0.20
!pip install supervision==0.15.0
!pip install torch torchvision
!pip install opencv-python
!pip install pandas matplotlib seaborn
!pip install scikit-learn
!pip install tensorflow
!pip install gdown
!pip install lap
!pip install filterpy
!pip install deep-sort-realtime

ERROR: Ignored the following yanked versions: 8.0.129, 8.0.174, 8.0.177, 8.1.21, 8.1.31, 8.2.7, 8.2.47
ERROR: Ignored the following versions that require a different python version: 8.0.10 Requires-Python >=3.7,<=3.11; 8.0.11 Requires-Python >=3.7,<=3.11; 8.0.12 Requires-Python >=3.7,<=3.11; 8.0.13 Requires-Python >=3.7,<=3.11; 8.0.14 Requires-Python >=3.7,<=3.11; 8.0.15 Requires-Python >=3.7,<=3.11; 8.0.16 Requires-Python >=3.7,<=3.11; 8.0.17 Requires-Python >=3.7,<=3.11; 8.0.18 Requires-Python >=3.7,<=3.11; 8.0.19 Requires-Python >=3.7,<=3.11; 8.0.20 Requires-Python >=3.7,<=3.11; 8.0.21 Requires-Python >=3.7,<=3.11; 8.0.22 Requires-Python >=3.7,<=3.11; 8.0.23 Requires-Python >=3.7,<=3.11; 8.0.24 Requires-Python >=3.7,<=3.11; 8.0.25 Requires-Python >=3.7,<=3.11; 8.0.26 Requires-Python >=3.7,<=3.11; 8.0.27 Requires-Python >=3.7,<=3.11; 8.0.28 Requires-Python >=3.7,<=3.11; 8.0.29 Requires-Python >=3.7,<=3.11; 8.0.30 Requires-Python >=3.7,<=3.11; 8.0.31 Requires-Python >=3.7,<=3.11; 8.0.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Cell 2: استيراد المكتبات اللازمة
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab import drive
import torch
import supervision as sv
from pathlib import Path
from IPython.display import display, HTML
from tqdm.notebook import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# Cell 3: تهيئة الإعدادات
# تثبيت Google Drive للوصول إلى البيانات
drive.mount('/content/drive')

# إنشاء مجلدات لتخزين البيانات والنماذج
MODELS_DIR = '/content/models'
DATA_DIR = '/content/data'
VIDEOS_DIR = '/content/videos'
FRAMES_DIR = '/content/frames'
TRACKS_DIR = '/content/tracks'
RESULTS_DIR = '/content/results'

for directory in [MODELS_DIR, DATA_DIR, VIDEOS_DIR, FRAMES_DIR, TRACKS_DIR, RESULTS_DIR]:
    os.makedirs(directory, exist_ok=True)

In [ ]:
# Cell 4: تنزيل مقاطع فيديو لمباريات برشلونة من YouTube
def download_youtube_video(youtube_id, output_path):
    """تنزيل فيديو من YouTube باستخدام youtube-dl"""
    !pip install -q yt-dlp
    !yt-dlp -f 'bestvideo[ext=mp4][height<=720]+bestaudio[ext=m4a]/best[ext=mp4]/best' \
        -o {output_path} \
        https://www.youtube.com/watch?v={youtube_id}

# قائمة بمعرفات فيديوهات لمباريات برشلونة على YouTube
# يمكن تحديث هذه القائمة بمباريات أخرى
BARCELONA_MATCHES = {
    'barca_vs_real_madrid': 'aMaW5JtIuJA',  # برشلونة ضد ريال مدريد
    'barca_vs_atletico': 'hKKXGiEFMBbltV88',     # برشلونة ضد أتلتيكو مدريد
    'barca_vs_sevilla': 'EnZ0JfD_U4c'       # برشلونة ضد إشبيلية
}

# تنزيل الفيديوهات
for match_name, youtube_id in BARCELONA_MATCHES.items():
    output_path = f"{VIDEOS_DIR}/{match_name}.mp4"
    if not os.path.exists(output_path):
        print(f"تنزيل {match_name}...")
        download_youtube_video(youtube_id, output_path)
    else:
        print(f"الفيديو {match_name} موجود بالفعل")

# بديلاً عن ذلك: تنزيل مجموعة بيانات SoccerNet للمباريات المسجلة
def download_soccernet_dataset():
    """تنزيل عينة من مجموعة بيانات SoccerNet"""
    !pip install -q gdown
    # رابط لعينة من مجموعة بيانات SoccerNet (تنزيل مباشر من Google Drive)
    !gdown --id 1zBVATJCX9KeUQABRr0e-3Jfuue8WvQR3 -O {DATA_DIR}/soccernet_sample.zip
    !unzip -q {DATA_DIR}/soccernet_sample.zip -d {DATA_DIR}

# تنزيل مجموعة البيانات (يمكن تعليق هذا السطر إذا كنت تستخدم فيديوهات YouTube فقط)
# download_soccernet_dataset()

In [ ]:
# Cell 1: تثبيت الأدوات المطلوبة
!pip install -q yt-dlp

# Cell 2: استيراد المكتبات
import os
from google.colab import drive

# Cell 3: تهيئة الإعدادات
# تثبيت Google Drive للوصول إلى البيانات
drive.mount('/content/drive')

# إنشاء مجلدات لتخزين البيانات والنماذج
MODELS_DIR = '/content/models'
DATA_DIR = '/content/data'
VIDEOS_DIR = '/content/videos'
FRAMES_DIR = '/content/frames'
TRACKS_DIR = '/content/tracks'
RESULTS_DIR = '/content/results'

for directory in [MODELS_DIR, DATA_DIR, VIDEOS_DIR, FRAMES_DIR, TRACKS_DIR, RESULTS_DIR]:
    os.makedirs(directory, exist_ok=True)

# Cell 4: دالة لتحميل الفيديوهات من يوتيوب
def download_youtube_video(youtube_id, output_path):
    """تنزيل فيديو من YouTube باستخدام yt-dlp"""
    !yt-dlp -f 'bestvideo[ext=mp4][height<=720]+bestaudio[ext=m4a]/best[ext=mp4]/best' \
        -o {output_path} \
        https://www.youtube.com/watch?v={youtube_id}

# Cell 5: روابط مباريات برشلونة (شغالة)
BARCELONA_MATCHES = {
    'barca_vs_real_madrid_2025': 'YsWzugAnsBw',     # برشلونة 5-2 ريال مدريد - نهائي السوبر 2025
    'barca_vs_real_madrid_2023': 'lPRW7d0B3BE',     # ريال مدريد 3-2 برشلونة - الدوري الإسباني 2023
    'barca_vs_real_madrid_friendly_2023': 'TFRpUANNVeg'  # برشلونة 3-0 ريال مدريد - ودية 2023
}

# Cell 6: تحميل الفيديوهات
for match_name, youtube_id in BARCELONA_MATCHES.items():
    output_path = f"{VIDEOS_DIR}/{match_name}.mp4"
    if not os.path.exists(output_path):
        print(f"تنزيل {match_name}...")
        download_youtube_video(youtube_id, output_path)
    else:
        print(f"الفيديو {match_name} موجود بالفعل")


In [ ]:
#Cell 5: تنزيل وتهيئة نموذج YOLOv8 للكشف عن اللاعبين
def setup_yolo_model():
    """تنزيل وتهيئة نموذج YOLOv8 للكشف عن الأشخاص"""
    # تنزيل النموذج المدرب مسبقًا YOLOv8x
    model = YOLO('yolov8x.pt')
    return model

# تهيئة نموذج YOLOv8
yolo_model = setup_yolo_model()

100%|██████████| 131M/131M [00:00<00:00, 284MB/s]


In [ ]:
# Cell 6: تهيئة نظام التتبع DeepSORT
def setup_tracker():
    """تهيئة متتبع DeepSORT"""
    tracker = DeepSort(
        max_age=30,  # عدد الإطارات التي سيستمر فيها المسار بدون اكتشاف
        n_init=3,   # عدد الإطارات اللازمة لتأكيد المسار
        nms_max_overlap=1.0,
        max_cosine_distance=0.3,
        nn_budget=100
    )
    return tracker

In [ ]:
# Cell 7: استخراج وتتبع اللاعبين من فيديو
def process_video(video_path, output_dir, max_frames=1000):
    """
    معالجة الفيديو للكشف عن اللاعبين وتتبعهم

    المدخلات:
        video_path: مسار الفيديو
        output_dir: دليل حفظ النتائج
        max_frames: الحد الأقصى للإطارات للمعالجة (للتطوير السريع)

    المخرجات:
        df_tracks: إطار بيانات يحتوي على مسارات اللاعبين
    """
    print(f"معالجة الفيديو: {video_path}")
    video_name = os.path.basename(video_path).split('.')[0]

    # إنشاء دليل للإطارات المعالجة لهذا الفيديو
    frames_dir = os.path.join(FRAMES_DIR, video_name)
    os.makedirs(frames_dir, exist_ok=True)

    # إنشاء مجلد لتخزين نتائج التتبع
    tracks_path = os.path.join(TRACKS_DIR, f"{video_name}_tracks.csv")

    # التحقق مما إذا كانت النتائج موجودة بالفعل
    if os.path.exists(tracks_path):
        print(f"تم العثور على نتائج التتبع المخزنة مسبقًا: {tracks_path}")
        return pd.read_csv(tracks_path)

    # فتح الفيديو
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # تهيئة المتتبع
    tracker = setup_tracker()

    # قائمة لتخزين بيانات المسارات
    tracks_data = []

    # معالجة الإطارات
    frame_idx = 0
    progress_bar = tqdm(total=min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), max_frames))

    while cap.isOpened() and frame_idx < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # اكتشاف الأشخاص باستخدام YOLOv8
        results = yolo_model(frame, classes=[0])  # الفئة 0 هي "شخص"

        detections = []
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                cls = box.cls[0].cpu().numpy()

                # تنسيق DeepSORT: [x1, y1, x2, y2, confidence]
                detection = [int(x1), int(y1), int(x2), int(y2), float(conf)]
                detections.append(detection)

        # تحديث المتتبع بالاكتشافات الجديدة
        tracks = tracker.update_tracks(detections, frame=frame)

        # احفظ الإطار بالاكتشافات (للتحقق البصري)
        if frame_idx % 10 == 0:  # حفظ كل 10 إطارات
            annotated_frame = frame.copy()

            # رسم المستطيلات والمعرفات على الإطار
            for track in tracks:
                if not track.is_confirmed():
                    continue

                track_id = track.track_id
                ltrb = track.to_ltrb()

                # رسم المستطيل والمعرف
                cv2.rectangle(annotated_frame,
                              (int(ltrb[0]), int(ltrb[1])),
                              (int(ltrb[2]), int(ltrb[3])),
                              (0, 255, 0), 2)
                cv2.putText(annotated_frame,
                            f"ID: {track_id}",
                            (int(ltrb[0]), int(ltrb[1] - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (0, 255, 0), 2)

            # حفظ الإطار المشروح
            frame_path = os.path.join(frames_dir, f"frame_{frame_idx:04d}.jpg")
            cv2.imwrite(frame_path, annotated_frame)

        # استخراج بيانات المسار لكل لاعب
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            ltrb = track.to_ltrb()

            # حساب المركز
            center_x = (ltrb[0] + ltrb[2]) / 2
            center_y = (ltrb[1] + ltrb[3]) / 2

            # تخزين بيانات المسار
            tracks_data.append({
                'frame': frame_idx,
                'track_id': track_id,
                'x': center_x / width,  # تطبيع للعرض
                'y': center_y / height,  # تطبيع للارتفاع
                'width': (ltrb[2] - ltrb[0]) / width,  # عرض الصندوق المطبع
                'height': (ltrb[3] - ltrb[1]) / height,  # ارتفاع الصندوق المطبع
                'time': frame_idx / fps,  # الوقت بالثواني
            })

        frame_idx += 1
        progress_bar.update(1)

    cap.release()
    progress_bar.close()

    # تحويل البيانات إلى إطار بيانات وحفظها
    df_tracks = pd.DataFrame(tracks_data)
    df_tracks.to_csv(tracks_path, index=False)

    print(f"تم معالجة {frame_idx} إطارات وتتبع {df_tracks['track_id'].nunique()} لاعب")
    return df_tracks


In [ ]:
# Cell 8: تحليل بيانات المسار وإعداد البيانات للتدريب
def analyze_player_movements(df_tracks, video_name):
    """
    تحليل حركات اللاعبين واستخراج الميزات ذات الصلة

    المدخلات:
        df_tracks: إطار بيانات يحتوي على بيانات المسار
        video_name: اسم الفيديو للتسمية

    المخرجات:
        df_features: إطار بيانات بميزات الحركة
    """
    print(f"تحليل حركات اللاعبين في {video_name}...")

    # تصفية المسارات القصيرة جدًا
    min_track_length = 50  # الحد الأدنى لطول المسار
    track_lengths = df_tracks.groupby('track_id').size()
    valid_tracks = track_lengths[track_lengths >= min_track_length].index
    df_filtered = df_tracks[df_tracks['track_id'].isin(valid_tracks)]

    # استخراج ميزات الحركة
    features = []

    for track_id, group in df_filtered.groupby('track_id'):
        # ترتيب حسب الإطار
        group = group.sort_values('frame')

        # حساب السرعة (التغير في الموضع)
        group['dx'] = group['x'].diff()
        group['dy'] = group['y'].diff()
        group['speed'] = np.sqrt(group['dx']**2 + group['dy']**2)

        # حساب التسارع
        group['acceleration'] = group['speed'].diff()

        # حساب الاتجاه (بالراديان)
        group['direction'] = np.arctan2(group['dy'], group['dx'])

        # تقسيم البيانات إلى مقاطع زمنية
        window_size = 15  # عدد الإطارات في النافذة الزمنية
        stride = 5       # خطوة التحريك

        for i in range(0, len(group) - window_size, stride):
            window = group.iloc[i:i+window_size]

            # حساب الإحصاءات في هذه النافذة
            feature_dict = {
                'track_id': track_id,
                'video': video_name,
                'start_frame': window['frame'].iloc[0],
                'end_frame': window['frame'].iloc[-1],
                'mean_x': window['x'].mean(),
                'mean_y': window['y'].mean(),
                'std_x': window['x'].std(),
                'std_y': window['y'].std(),
                'mean_speed': window['speed'].mean(),
                'max_speed': window['speed'].max(),
                'mean_acceleration': window['acceleration'].mean(),
                'direction_change': window['direction'].diff().abs().sum(),
                # الحركة في الاتجاه X و Y
                'x_displacement': window['x'].iloc[-1] - window['x'].iloc[0],
                'y_displacement': window['y'].iloc[-1] - window['y'].iloc[0],
                # إضافة تسلسل الإحداثيات X و Y للاستخدام في LSTM
                'x_sequence': window['x'].tolist(),
                'y_sequence': window['y'].tolist(),
                'speed_sequence': window['speed'].tolist(),
            }

            features.append(feature_dict)

    # تحويل إلى إطار بيانات
    df_features = pd.DataFrame(features)

    # حفظ الميزات
    features_path = os.path.join(DATA_DIR, f"{video_name}_features.csv")
    df_features.drop(['x_sequence', 'y_sequence', 'speed_sequence'], axis=1).to_csv(features_path, index=False)

    return df_features

# Cell 9: تصنيف نمط اللعب وتجميع البيانات
def classify_playing_patterns(df_features, num_clusters=4):
    """
    تصنيف أنماط اللعب باستخدام التجميع

    المدخلات:
        df_features: إطار بيانات بميزات الحركة
        num_clusters: عدد المجموعات

    المخرجات:
        df_features: إطار بيانات مع تصنيفات النمط
    """
    from sklearn.cluster import KMeans

    # تحديد الميزات للتجميع
    clustering_features = [
        'mean_x', 'mean_y', 'std_x', 'std_y',
        'mean_speed', 'max_speed', 'mean_acceleration',
        'direction_change', 'x_displacement', 'y_displacement'
    ]

    # تطبيع البيانات
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_features[clustering_features])

    # التجميع بـ K-means
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    df_features['pattern_cluster'] = kmeans.fit_predict(scaled_features)

    # توصيف المجموعات
    cluster_descriptions = {
        0: "هجوم سريع",
        1: "تراجع دفاعي",
        2: "استحواذ وسط الملعب",
        3: "تحرك جانبي"
    }

    # إضافة وصف المجموعة
    df_features['pattern_description'] = df_features['pattern_cluster'].map(cluster_descriptions)

    return df_features

In [ ]:
#Cell 10: إعداد بيانات التدريب لنموذج LSTM
def prepare_lstm_data(df_features):
    """
    إعداد البيانات للتدريب على نموذج LSTM

    المدخلات:
        df_features: إطار بيانات بميزات الحركة

    المخرجات:
        X_train, X_test, y_train, y_test: بيانات التدريب والاختبار
    """
    # تحضير المدخلات (تسلسلات x, y) والمخرجات (تسلسلات مستقبلية)
    X = []
    y = []

    sequence_length = 10  # طول تسلسل الإدخال
    prediction_steps = 5  # عدد الخطوات المستقبلية للتنبؤ

    for _, row in df_features.iterrows():
        x_seq = row['x_sequence']
        y_seq = row['y_sequence']

        if len(x_seq) < sequence_length + prediction_steps:
            continue

        # استخدام تسلسل الإدخال للتنبؤ بالخطوات المستقبلية
        for i in range(len(x_seq) - sequence_length - prediction_steps):
            x_input = []
            for j in range(sequence_length):
                x_input.append([x_seq[i+j], y_seq[i+j]])

            x_output = []
            for j in range(prediction_steps):
                x_output.append([x_seq[i+sequence_length+j], y_seq[i+sequence_length+j]])

            X.append(x_input)
            y.append(x_output)

    # تحويل إلى مصفوفات NumPy
    X = np.array(X)
    y = np.array(y)

    # تقسيم إلى مجموعات تدريب واختبار
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"شكل بيانات التدريب X: {X_train.shape}, y: {y_train.shape}")
    print(f"شكل بيانات الاختبار X: {X_test.shape}, y: {y_test.shape}")

    return X_train, X_test, y_train, y_test

# Cell 11: بناء وتدريب نموذج LSTM للتنبؤ بحركات اللاعبين
def build_lstm_model(X_train, y_train, X_test, y_test):
    """
    بناء وتدريب نموذج LSTM للتنبؤ بحركات اللاعبين

    المدخلات:
        X_train, y_train: بيانات التدريب
        X_test, y_test: بيانات الاختبار

    المخرجات:
        model: النموذج المدرب
    """
    input_shape = X_train.shape[1:]
    output_shape = y_train.shape[1:]

    # تحقق من وجود نموذج محفوظ مسبقًا
    model_path = os.path.join(MODELS_DIR, 'lstm_player_prediction.h5')
    if os.path.exists(model_path):
        print("تحميل نموذج LSTM محفوظ مسبقًا...")
        model = load_model(model_path)
        return model

    # بناء نموذج LSTM
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(np.prod(output_shape))
    ])

    # إعادة تشكيل طبقة الإخراج
    model.add(tf.keras.layers.Reshape(output_shape))

    # تجميع النموذج
    model.compile(
        optimizer='adam',
        loss='mse'
    )

    # ملخص النموذج
    model.summary()

    # تدريب النموذج
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=30,
        batch_size=32,
        callbacks=[early_stopping, model_checkpoint]
    )

    # تحليل أداء النموذج
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='خسارة التدريب')
    plt.plot(history.history['val_loss'], label='خسارة التحقق')
    plt.title('أداء نموذج التنبؤ LSTM')
    plt.xlabel('الحقبة (Epoch)')
    plt.ylabel('متوسط مربع الخطأ (MSE)')
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR, 'lstm_training_history.png'))

    return model


In [ ]:
# Cell 12: تقييم نموذج LSTM
def evaluate_lstm_model(model, X_test, y_test):
    """
    تقييم نموذج LSTM على بيانات الاختبار

    المدخلات:
        model: نموذج LSTM المدرب
        X_test, y_test: بيانات الاختبار
    """
    # التنبؤ على بيانات الاختبار
    y_pred = model.predict(X_test)

    # حساب متوسط مربع الخطأ لكل خطوة زمنية
    mse_by_step = np.mean((y_test - y_pred)**2, axis=(0, 2))

    # رسم بياني للخطأ حسب الخطوة الزمنية
    plt.figure(figsize=(10, 5))
    plt.plot(mse_by_step, marker='o')
    plt.title('دقة التنبؤ حسب الخطوة الزمنية المستقبلية')
    plt.xlabel('خطوة التنبؤ')
    plt.ylabel('متوسط مربع الخطأ (MSE)')
    plt.grid(True)
    plt.savefig(os.path.join(RESULTS_DIR, 'prediction_accuracy_by_step.png'))

    # عرض مثال للتنبؤات
    num_examples = 5
    plt.figure(figsize=(15, 10))

    for i in range(num_examples):
        plt.subplot(num_examples, 1, i+1)

        # الموقع الفعلي
        actual_x = y_test[i, :, 0]
        actual_y = y_test[i, :, 1]

        # الموقع المتوقع
        pred_x = y_pred[i, :, 0]
        pred_y = y_pred[i, :, 1]

        # رسم بياني
        plt.plot(actual_x, actual_y, 'b-', marker='o', label='الموقع الفعلي')
        plt.plot(pred_x, pred_y, 'r--', marker='x', label='الموقع المتوقع')
        plt.title(f'مثال {i+1}: مقارنة بين الموقع الفعلي والمتوقع')
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'prediction_examples.png'))

In [ ]:
# Cell 13: تحليل الأنماط التكتيكية وتوليد التوصيات
def tactical_pattern_analysis(df_features):
    """
    تحليل الأنماط التكتيكية وتوليد توصيات للمدرب

    المدخلات:
        df_features: إطار بيانات بميزات الحركة وتصنيفات النمط
    """
    # تحليل توزيع الأنماط
    pattern_distribution = df_features['pattern_description'].value_counts(normalize=True) * 100

    plt.figure(figsize=(10, 6))
    pattern_distribution.plot(kind='bar', color='skyblue')
    plt.title('توزيع الأنماط التكتيكية')
    plt.xlabel('النمط التكتيكي')
    plt.ylabel('النسبة المئوية (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'tactical_pattern_distribution.png'))

    # تحليل السرعة حسب النمط
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='pattern_description', y='mean_speed', data=df_features)
    plt.title('متوسط السرعة حسب النمط التكتيكي')
    plt.xlabel('النمط التكتيكي')
    plt.ylabel('متوسط السرعة')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'speed_by_pattern.png'))

    # خريطة حرارية لمواقع اللاعبين حسب النمط
    plt.figure(figsize=(12, 8))
    for i, pattern in enumerate(pattern_distribution.index):
        plt.subplot(2, 2, i+1)
        pattern_data = df_features[df_features['pattern_description'] == pattern]

        plt.scatter(pattern_data['mean_x'], pattern_data['mean_y'],
                   alpha=0.3, s=10)
        plt.title(f'مواقع اللاعبين: {pattern}')
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.gca().set_aspect('equal')

        # إضافة خطوط تمثل حدود الملعب
        plt.axhline(y=0.5, color='black', linestyle='--', alpha=0.3)  # خط الوسط
        plt.axvline(x=0, color='black', alpha=0.3)  # خط المرمى اليسار
        plt.axvline(x=1, color='black', alpha=0.3)  # خط المرمى اليمين
        plt.axvline(x=0.5, color='black', linestyle='--', alpha=0.3)  # خط المنتصف
        plt.xlabel('الموقع الأفقي (x)')
        plt.ylabel('الموقع العمودي (y)')

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'heatmap_by_pattern.png'))
    plt.close()

In [ ]:
# 1. استخراج بيانات الحركة من التتبع (PBAS)
import numpy as np # Import numpy

def extract_motion_data(frames, contours_data):
    data = []
    for i, contours in enumerate(contours_data):
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            center_x, center_y = x + w // 2, y + h // 2
            data.append([i, center_x, center_y])
    return np.array(data)


# 2. تحضير بيانات LSTM
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def prepare_lstm_data(data, sequence_length=10):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, 1:])
        y.append(data[i + sequence_length, 1:])
    return np.array(X), np.array(y)

#  **Placeholder for frames and contours_data - Replace with your actual data**
frames = [] # Example: frames = [cv2.imread(frame_path) for frame_path in frame_paths]
contours_data = [] # Example: contours_data = [[get_contours(frame)] for frame in frames]

# استخراج بيانات الحركة
motion_data = extract_motion_data(frames, contours_data)

# تطبيع وتحضير البيانات
scaler = MinMaxScaler()
motion_data_scaled = scaler.fit_transform(motion_data)  # motion_data ناتجة من extract_motion_data
X, y = prepare_lstm_data(motion_data_scaled)


# 3. تدريب نموذج LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))
model.add(Dense(2))  # الإحداثيات (x, y)

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)


# 4. حفظ النتائج وتحليل الأداء
predicted = model.predict(X)
predicted_coords = scaler.inverse_transform(np.concatenate([np.zeros((predicted.shape[0], 1)), predicted], axis=1))[:, 1:]

# حفظ النتائج
np.save("predicted_coords.npy", predicted_coords)


# 5. واجهة Streamlit
# حفظ هذا الجزء في ملف streamlit_app.py لتشغيله بـ `streamlit run streamlit_app.py`

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

st.title("تحليل تحركات اللاعبين باستخدام LSTM")

# تحميل النتائج
predicted = np.load("predicted_coords.npy")

# عرض النتائج
st.subheader("المسار المتوقع للحركة")
fig, ax = plt.subplots()
ax.plot(predicted[:, 0], predicted[:, 1], label="LSTM Prediction", color='blue')
st.pyplot(fig)

# رفع فيديو للمباراة
uploaded_file = st.file_uploader("ارفع فيديو لمباراة حقيقية للمقارنة", type=["mp4"])

if uploaded_file is not None:
    st.video(uploaded_file)
    st.info("سيتم تحليل الفيديو لاحقاً وربطه مع النتائج لتقييم أداء النموذج.")

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.